In [1]:
# API Key를 환경변수로 관리하기 위한 설정 파일

from dotenv import load_dotenv

# API Key 정보로드
# - OPENAI_API_KEY = ""
# - LANGCHAIN_TRACING_V2 = "true"
# - LANGCHAIN_ENDPOINT = "https://api.smith.langchain.com"
# - LANGCHAIN_API_KEY = ""
load_dotenv()

import os
os.environ["LANGCHAIN_PROJECT"] = "langchain_study" # Langsmith project 명

In [2]:
# 비교대상 모델ID 상수 정의
IS_HOME = False

if IS_HOME:
    AYA = "aya:8b-23-q8_0"
    LLAMA3 = "llama3:8b-instruct-q8_0"
    GEMMA = "gemma:7b-instruct-q8_0"
    PHI3 = "phi3:instruct"
else:
    AYA = "aya"
    LLAMA3 = "llama3:instruct"
    GEMMA = "gemma"
    PHI3 = "phi3:instruct"

GPT = "gpt-4o"

In [3]:
from langchain_community.llms import Ollama

llm_aya = Ollama(model=AYA, temperature=0)
llm_llama3 = Ollama(model=LLAMA3, temperature=0)
llm_gemma = Ollama(model=GEMMA, temperature=0)
llm_phi3 = Ollama(model=PHI3, temperature=0)


In [4]:
from langchain_openai import ChatOpenAI

llm_gpt = ChatOpenAI(model=GPT, temperature=0)

In [5]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    System : 너는 다음 Instruction을 잘 수행하는 assistant 이다.
    Instruction : {instruction}
    """
)

In [6]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [7]:
chain_aya = prompt | llm_aya | output_parser
chain_llama3 = prompt | llm_llama3 | output_parser
chain_gemma = prompt | llm_gemma | output_parser
chain_phi3 = prompt | llm_phi3 | output_parser
chain_gpt = prompt | llm_gpt | output_parser

param = {
    AYA : chain_aya ,
    LLAMA3 : chain_llama3 ,
    GEMMA : chain_gemma ,
    PHI3 : chain_phi3 ,
    GPT : chain_gpt ,
}

from langchain_core.runnables import RunnableParallel

chain_llms = RunnableParallel(**param)

#### (TEST) 평가대상 LLM 문제 출제 및 답안 작성

In [44]:
# chain_llms 실행 Test
responses = chain_llms.invoke({"instruction":"식혜라는 음료수에 대해 알려주세요."})

from pprint import pprint
pprint(responses)

{'aya': '식혜는 한국의 전통 음료로, 쌀로 만든 시럽이나 꿀로 단맛을 내고 때때로 마늘이나 생강과 같은 향신료를 넣어 만듭니다. '
        '그것은 종종 차가운 상태로 제공되며, 달콤하고 약간 매콤한 맛이 납니다. 식혜는 한국 문화에서 인기 있는 음료이며, 특히 '
        '여름에 인기가 많습니다. 그것은 종종 건강 증진과 소화 개선에 도움이 되는 것으로 여겨집니다.',
 'gemma': '식혜는 한국에서 유명한 음료수입니다. 주로 과일, 특히 사과를 주원으로 하고, 설탕과 향이 들어간 음료입니다. 식혜는 '
          '건강에 도움이 되고, 특히 여름에 마셔면 신선감을 느끼고 휴식을 취할 수 있습니다.',
 'gpt-4o': '식혜는 한국의 전통 음료수로, 주로 쌀과 엿기름을 사용하여 만듭니다. 식혜는 달콤하고 시원한 맛이 특징이며, 특히 '
           '명절이나 특별한 행사에서 자주 즐겨 마십니다. 다음은 식혜에 대한 주요 정보입니다:\n'
           '\n'
           '1. **재료**:\n'
           '   - **쌀**: 주로 찹쌀을 사용하지만, 멥쌀을 사용하기도 합니다.\n'
           '   - **엿기름**: 엿기름 가루를 물에 우려내어 사용합니다.\n'
           '   - **설탕**: 단맛을 더하기 위해 사용합니다.\n'
           '   - **물**: 기본적인 재료로 사용됩니다.\n'
           '\n'
           '2. **만드는 방법**:\n'
           '   1. 엿기름 가루를 물에 넣고 잘 섞은 후, 체에 걸러 엿기름 물을 만듭니다.\n'
           '   2. 찹쌀을 씻어 물에 불린 후, 밥을 짓습니다.\n'
           '   3. 지은 밥을 엿기름 물에 넣고 일정 시간 동안 발효시킵니다. 이 과정에서 밥알이 떠오르면 발효가 잘 된 '
           '것입니다.\n'
         

In [8]:
#instruction = "1990년대 한국 대중가요에 대해 알려주세요."
instruction = "필리핀의 대표적인 대중교통 알려줘"

#### CASE # 1. StrOutputParser

In [47]:
eval_prompt = PromptTemplate(
    template="""
    System : 
    너는 llm model들의 답변을 비교하고 평가하는 AI 이다.
    Instruction과 Responses 안의 각각의 llm별 응답을 
    정확성(Accuracy), 관련성(Relevance), 유창성(Fluency), 완전성(Completeness) 측면에서    
    분석하고 최고 점수 5점으로 0점 ~ 5점 사이 점수를 부여하라.

    한국어로 답변해줘.

    Instruction : {instruction}
    Resonses : {responses}
    """,
    input_variables=["instruction", "responses"],
)

In [48]:
from langchain_core.runnables import RunnablePassthrough

chain_combinded = (
    {"responses" : chain_llms, "instruction" : RunnablePassthrough()}
    | eval_prompt
    | llm_gpt
    | output_parser
)

- 실행(invoke)

In [49]:
response = chain_combinded.invoke({"instruction":instruction})
print(response)

### 평가 기준
1. **정확성(Accuracy)**: 제공된 정보가 사실에 부합하는지 여부.
2. **관련성(Relevance)**: 질문에 대한 답변이 얼마나 관련성이 있는지.
3. **유창성(Fluency)**: 문장이 얼마나 자연스럽고 읽기 쉬운지.
4. **완전성(Completeness)**: 답변이 질문에 대해 얼마나 완전하게 설명하는지.

### 평가 결과

#### aya
- **정확성**: 5점
  - 쿠바 미사일 위기의 주요 사건과 결과를 정확하게 설명함.
- **관련성**: 5점
  - 질문에 대한 답변이 매우 관련성이 높음.
- **유창성**: 5점
  - 문장이 자연스럽고 읽기 쉬움.
- **완전성**: 5점
  - 사건의 배경, 전개, 결과를 모두 포함하여 완전하게 설명함.

#### llama3:instruct
- **정확성**: 5점
  - 쿠바 미사일 위기의 주요 사건과 결과를 정확하게 설명함.
- **관련성**: 5점
  - 질문에 대한 답변이 매우 관련성이 높음.
- **유창성**: 5점
  - 문장이 자연스럽고 읽기 쉬움.
- **완전성**: 5점
  - 사건의 배경, 전개, 결과를 모두 포함하여 완전하게 설명함.

#### gemma
- **정확성**: 2점
  - 쿠바 미사일 위기와 관련된 정보가 부정확하고, 쿠바의 외교 정책에 대한 설명이 주를 이룸.
- **관련성**: 2점
  - 질문에 대한 답변이 다소 관련성이 떨어짐.
- **유창성**: 4점
  - 문장은 자연스럽고 읽기 쉬움.
- **완전성**: 2점
  - 쿠바 미사일 위기의 주요 사건과 결과에 대한 설명이 부족함.

#### phi3:instruct
- **정확성**: 0점
  - 제공된 정보가 쿠바 미사일 위기와 전혀 관련이 없음.
- **관련성**: 0점
  - 질문에 대한 답변이 전혀 관련성이 없음.
- **유창성**: 1점
  - 문장이 반복적이고 읽기 어려움.
- **완전성**: 0점
  - 쿠바 미사일 위기에 대한 설명이 전혀 없음.

####

#### CASE # 2. PydanticOutputParser

In [9]:
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field

class EvaluationByModel(BaseModel):
    model_id: str = Field(description="LLM 모델 이름 또는 LLM 모델 ID")
    accuracy_eval: str = Field(description="정확성(Accuracy) 평가")
    accuracy_score: int = Field(description="정확성(Accuracy) 평가 점수")
    relevance_eval: str = Field(description="관련성(Relevance) 평가")
    relevance_score: int = Field(description="관련성(Relevance) 평가 점수")
    fluency_eval: str = Field(description="유창성(Fluency) 평가")
    fluency_score: int = Field(description="유창성(Fluency) 평가 점수")
    completeness_eval: str = Field(description="완전성(Completeness) 평가")
    completeness_score: int = Field(description="관련성(Relevance) 평가 점수")

class EvaluationResponse(BaseModel):
    instruction: str = Field(description="Instruction 내용 원문")
    accuracy: str = Field(description="정확성(Accuracy) 평가 기준")
    relevance: str = Field(description="관련성(Relevance) 평가 기준")
    fluency: str = Field(description="유창성(Fluency) 평가 기준")
    completeness: str = Field(description="완전성(Completeness) 평가 기준")
    evaluation_by_model: List[EvaluationByModel] = Field(description="LLM 모델별 세부 평가 내용")
    overall: str = Field(description="종합평가")

In [10]:
# Schema 확인
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
f = convert_pydantic_to_openai_function(EvaluationResponse)
f

c:\Users\prof\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.3.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


{'name': 'EvaluationResponse',
 'description': '',
 'parameters': {'type': 'object',
  'properties': {'instruction': {'description': 'Instruction 내용 원문',
    'type': 'string'},
   'accuracy': {'description': '정확성(Accuracy) 평가 기준', 'type': 'string'},
   'relevance': {'description': '관련성(Relevance) 평가 기준', 'type': 'string'},
   'fluency': {'description': '유창성(Fluency) 평가 기준', 'type': 'string'},
   'completeness': {'description': '완전성(Completeness) 평가 기준',
    'type': 'string'},
   'evaluation_by_model': {'description': 'LLM 모델별 세부 평가 내용',
    'type': 'array',
    'items': {'type': 'object',
     'properties': {'model_id': {'description': 'LLM 모델 이름 또는 LLM 모델 ID',
       'type': 'string'},
      'accuracy_eval': {'description': '정확성(Accuracy) 평가', 'type': 'string'},
      'accuracy_score': {'description': '정확성(Accuracy) 평가 점수',
       'type': 'integer'},
      'relevance_eval': {'description': '관련성(Relevance) 평가', 'type': 'string'},
      'relevance_score': {'description': '관련성(Relevance)

In [11]:
from langchain.output_parsers import PydanticOutputParser

p_parser = PydanticOutputParser(pydantic_object=EvaluationResponse)

In [12]:
eval_prompt = PromptTemplate(
    template="""
    System : 
    너는 llm model들의 답변을 비교하고 평가하는 AI 이다.
    Instruction과 Responses 안의 각각의 llm별 응답을
    정확성(Accuracy), 관련성(Relevance), 유창성(Fluency), 완전성(Completeness) 측면에서    
    분석하고 최고 점수 5점으로 0점 ~ 5점 사이 점수를 부여하라.

    한국어로 답변해줘.
    Format 에 맞춰서 답변해줘

    Instruction : {instruction}
    Responses : {responses}

    Format : 
    {format}
    """,
    input_variables=["instruction", "responses"],
    partial_variables={"format" : p_parser.get_format_instructions()},
)

In [13]:
from langchain_core.runnables import RunnablePassthrough

chain_combinded = (
    {"responses" : chain_llms, "instruction" : RunnablePassthrough()}
    | eval_prompt 
    | llm_gpt
    | p_parser
)

- 실행(invoke)

In [14]:
response = chain_combinded.invoke({"instruction":instruction})
print(response)

c:\Users\prof\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '70.10.15.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\prof\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '70.10.15.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\prof\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '70.10.15.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings


instruction='필리핀의 대표적인 대중교통 알려줘' accuracy='각 모델의 응답이 필리핀의 대표적인 대중교통 수단을 정확하게 설명하는지 평가' relevance='응답이 주어진 질문과 얼마나 관련성이 있는지 평가' fluency='응답이 문법적으로 얼마나 유창하게 작성되었는지 평가' completeness='응답이 질문에 대해 얼마나 완전하게 답변했는지 평가' evaluation_by_model=[EvaluationByModel(model_id='aya', accuracy_eval='지프니, 버스, 트라이시클, 보트, 기차 등 필리핀의 주요 대중교통 수단을 정확하게 설명함', accuracy_score=5, relevance_eval='질문과 매우 관련성이 높음', relevance_score=5, fluency_eval='문법적으로 매우 유창하게 작성됨', fluency_score=5, completeness_eval='다양한 대중교통 수단을 포괄적으로 설명함', completeness_score=5), EvaluationByModel(model_id='llama3:instruct', accuracy_eval='지프니, 버스, 트라이시클, 택시, 페디캡, 페리 등 필리핀의 주요 대중교통 수단을 정확하게 설명함', accuracy_score=5, relevance_eval='질문과 매우 관련성이 높음', relevance_score=5, fluency_eval='문법적으로 매우 유창하게 작성됨', fluency_score=5, completeness_eval='다양한 대중교통 수단을 포괄적으로 설명함', completeness_score=5), EvaluationByModel(model_id='gemma', accuracy_eval='LRT, MRT, 버스, 지프니 등 필리핀의 주요 대중교통 수단을 설명함', accuracy_score=4, relevance_eval='질문과 관련성이 높음', relevance_score=4, fluency_eva

c:\Users\prof\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '70.10.15.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


#### 질문과 평가결과를 DB에 저장하기

In [15]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session, relationship, sessionmaker

Base = declarative_base()

engine = create_engine("sqlite://", echo=True)

C:\Users\tiffanie.kim\AppData\Local\Temp\ipykernel_12772\2944376351.py:5: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


- CASE #1. 명시적으로 Table 만들기

In [60]:
from sqlalchemy import Column, Integer, String, Text, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker

Base = declarative_base()

class TEvaluationResponse(Base):
    __tablename__ = 'evaluation_responses'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    instruction = Column(Text, nullable=False)
    accuracy = Column(Text, nullable=False)
    relevance = Column(Text, nullable=False)
    fluency = Column(Text, nullable=False)
    completeness = Column(Text, nullable=False)
    overall = Column(Text, nullable=False)
    
    evaluations = relationship("TEvaluationByModel", back_populates="response")

class TEvaluationByModel(Base):
    __tablename__ = 'evaluation_by_model'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    model_id = Column(String, nullable=False)
    accuracy_eval = Column(Text, nullable=False)
    accuracy_score = Column(Integer, nullable=False)
    relevance_eval = Column(Text, nullable=False)
    relevance_score = Column(Integer, nullable=False)
    fluency_eval = Column(Text, nullable=False)
    fluency_score = Column(Integer, nullable=False)
    completeness_eval = Column(Text, nullable=False)
    completeness_score = Column(Integer, nullable=False)
    
    response_id = Column(Integer, ForeignKey('evaluation_responses.id'))
    response = relationship("TEvaluationResponse", back_populates="evaluations")

# Create an engine and a session
engine = create_engine('sqlite:///evaluations.db')
Session = sessionmaker(bind=engine)
Base.metadata.create_all(engine)


C:\Users\tiffanie.kim\AppData\Local\Temp\ipykernel_2876\1723458513.py:5: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


- CASE #2. Pydantic Schema를 이용하여 동적으로 Table 만들기 (ChatGPT가 만들어 줌)

In [16]:
from sqlalchemy import Column, Integer, String, Text, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from typing import Dict, Type

Base = declarative_base()

# Mapping Pydantic types to SQLAlchemy types
type_mapping = {
    str: String,
    int: Integer,
    float: Text,
}

def create_sqlalchemy_model(pydantic_model: Type[BaseModel], base: Type[Base], table_name: str, exclude_fields: List[str] = None, additional_columns: Dict[str, Type[Column]] = None):
    if exclude_fields is None:
        exclude_fields = []
    if additional_columns is None:
        additional_columns = {}

    fields = pydantic_model.__fields__
    columns = {
        '__tablename__': table_name,
        'id': Column(Integer, primary_key=True, autoincrement=True)
    }
    
    for field_name, field in fields.items():
        if field_name in exclude_fields:
            continue
        field_type = field.outer_type_
        columns[field_name] = Column(type_mapping[field_type], nullable=False)
    
    # Add additional columns for relationships or other special cases
    columns.update(additional_columns)
    
    # Create the new model class
    model = type(table_name, (base,), columns)
    return model

# Dynamically create SQLAlchemy models without relationships
TEvaluationResponse = create_sqlalchemy_model(
    EvaluationResponse, 
    Base, 
    'evaluation_responses', 
    exclude_fields=['evaluation_by_model']
)
TEvaluationByModel = create_sqlalchemy_model(
    EvaluationByModel, 
    Base, 
    'evaluation_by_model'
)

# Define relationships after both classes have been created
TEvaluationResponse.evaluations = relationship("TEvaluationByModel", back_populates="response")
TEvaluationByModel.response_id = Column(Integer, ForeignKey('evaluation_responses.id'))
TEvaluationByModel.response = relationship("TEvaluationResponse", back_populates="evaluations")

# Create an engine and a session
engine = create_engine('sqlite:///evaluations.db')
Session = sessionmaker(bind=engine)
Base.metadata.create_all(engine)


In [17]:
def save_evaluation_response(session, evaluation_response):

    response_dict = evaluation_response.dict(exclude={"evaluation_by_model"})
    response = TEvaluationResponse(
        **response_dict  # Unpack the dictionary into keyword arguments
    )
    session.add(response)
    session.flush()

    for eval_model in evaluation_response.evaluation_by_model:
        model_evaluation = TEvaluationByModel(
            response_id=response.id,  # Use the generated ID
            **eval_model.dict()  # Unpack the dictionary into keyword arguments
        )
        session.add(model_evaluation)
    
    session.commit()

In [20]:
TEvaluationResponse.__dict__

mappingproxy({'__tablename__': 'evaluation_responses',
              'id': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x2246aa10f50>,
              'instruction': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x2246aa10fa0>,
              'accuracy': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x2246aa10ff0>,
              'relevance': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x2246aa11040>,
              'fluency': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x2246aa11090>,
              'completeness': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x2246aa110e0>,
              'overall': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x2246aa11130>,
              'evaluations': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x2246a9cf1b0>,
              '__module__': '__main__',
              '__doc__': None,
              '_sa_class_manager': <ClassManager of <class '__main__.evaluation_responses'> at 2245ba9a030>,
          

In [18]:
session = Session()
save_evaluation_response(session, response)
session.close()

InvalidRequestError: When initializing mapper mapped class evaluation_responses->evaluation_responses, expression 'TEvaluationByModel' failed to locate a name ('TEvaluationByModel'). If this is a class name, consider adding this relationship() to the <class '__main__.evaluation_responses'> class after both dependent classes have been defined.